In [1]:
import requests
import pandas as pd

# Endpoint JSON que a página da B3 usa internamente
url = "https://www.infomoney.com.br/cotacoes/empresas-b3/"

# Faz a requisição
response = requests.get(url)

html = response.text

In [2]:
from bs4 import BeautifulSoup
import pandas as pd

# Suponha que `html` seja o conteúdo da página
soup = BeautifulSoup(html, 'html.parser')

# Extrai todos os elementos relevantes em ordem
todos = soup.find_all(["div", "td"])

dados = []
setor_atual = None
i = 0

while i < len(todos):
    el = todos[i]
    
    # Verifica se é header de setor
    if el.name == "div" and "header-sector" in el.get("class", []):
        h2 = el.find("h2")
        if h2:
            setor_atual = h2.text.strip()
        i += 1
        continue

    # Verifica se é um <td class="higher">
    if el.name == "td" and "higher" in el.get("class", []):
        empresa = el.text.strip()
        tickers = []

        # Pega os próximos td strong (ou para se encontrar outro header-sector ou higher)
        j = i + 1
        while j < len(todos):
            next_el = todos[j]
            if next_el.name == "div" and "header-sector" in next_el.get("class", []):
                break
            if next_el.name == "td" and "higher" in next_el.get("class", []):
                break
            if next_el.name == "td" and "strong" in next_el.get("class", []):
                ticker = next_el.text.strip()
                if not ticker.endswith("F"):
                    tickers.append(ticker)
            j += 1

        # Salva os dados: Setor + Empresa + Tickes
        dados.append([setor_atual, empresa] + tickers)
        i = j
    else:
        i += 1

# Define o número máximo de tickers para nomear as colunas
max_tickers = max(len(linha) - 2 for linha in dados)
colunas = ["Setor", "Empresa"] + [f"Ticker_{i+1}" for i in range(max_tickers)]

# Cria o DataFrame final
df = pd.DataFrame(dados, columns=colunas)

df = df.sort_values(by='Empresa').reset_index(drop=True)
df.to_excel('Lista_Empresas_B3.xlsx')

In [25]:
df_filtrado = df.query("Setor != 'Financeiro'")
df_filtrado = df_filtrado[~df_filtrado['Empresa'].str.contains("FII|Banco|Invest|Recebiveis|Recebíveis|BKR|Agro|AGRO|S&P|Sp500|FT|Inc", case=False, na=False)]

df_filtrado

,Setor,Empresa,Ticker_1,Ticker_2,Ticker_3,Ticker_4
1,Bens Industriais,3R Petroleum,RRRP3,None,None,None
2,Bens Industriais,3tentos,TTEN3,None,None,None
3,Utilidade Pública,AES Brasil,AESB3,None,None,None
5,Consumo Cíclico,AZZAS 2154 S.A.,AZZA3,None,None,None
6,Saúde,Abbott Laboratories,ABTT34,None,None,None
...,...,...,...,...,...,...
423,Tecnologia da Informação,Xerox,XRXB34,None,None,None
424,Consumo Cíclico,YDUQS,YDUQ3,None,None,None
425,Bens Industriais,ZAMP S.A.,ZAMP3,None,None,None
428,Bens Industriais,d1000,DMVF3,None,None,None


In [27]:
# Cria o dicionário de empresas com nomes formatados e links vazios
empresas = {
    nome.upper().replace(" ", "_"): ""
    for nome in df_filtrado['Empresa']
}

In [23]:
empresas = {'3TENTOS': 'https://ri.3tentos.com.br/informacoes-finaceiras/central-de-resultados/',
            'AFLUENTE_T': 'https://ri.neoenergia.com/resultados-e-indicadores/central-de-resultados/',
            'AGRIBRASIL': 'https://www.agribrasil.net/informacoes-aos-investidores/central-de-resultados/',
            'ALLIAR': 'https://ri.allianca.com/informacoes-financeiras/central-de-resultados/',
            'ALLIED_TECNOLOGIA': 'https://ri.alliedbrasil.com.br/informacoes-financeiras-e-operacionais/central-de-resultados/',
            'ALLOS': 'https://ri.allos.co/informacoes-financeiras/central-de-resultados/',
            'ALPARGATAS': 'https://ri.alpargatas.com.br/informacoes-aos-investidores/resultados/',
            'ALPHAVILLE': 'https://ri.alphaville.com.br/informacoes-financeiras/central-de-resultados/',
            'AMBIPAR': 'https://ri.ambipar.com/informacoes-financeiras/resultados-trimestrais/',
            'ANIMA': 'https://ri.animaeducacao.com.br/listresultados.aspx?idCanal=PpIsKQGpxHMOqAu5XCUCKQ==',
            'AREZZO': 'https://ri.azzas2154.com.br/resultados-e-apresentacoes/central-de-resultados/',
            'ARMAC': 'https://ri.armac.com.br/informacoes-financeiras/central-de-resultados/',
            'ASSAÍ': 'https://ri.assai.com.br/informacoes-financeiras/resultados-trimestrais/',
            'ATMA_PARTICIPAÇÕES': 'https://www.atmasa.com.br/informacoes-financeiras/resultados-trimestrais/',
            'AUREN_ENERGIA': 'https://ri.aurenenergia.com.br/informacoes-financeiras/central-de-resultados-auren/',
            'AUTOMOB': 'https://ri.automob.com.br/divulgacoes-e-documentos/central-de-resultados/',
            'AVON': 'https://ri.naturaeco.com/resultados-e-apresentacoes/central-de-resultados/',
            'BR_PARTNERS': 'https://ri.brpartners.com.br/informacoes-publicas/central-de-resultados/',
            'BEMOBI': 'https://ri.bemobi.com.br/informacoes-financeiras/resultados-trimestrais/',
            'BIOMM': 'https://ri.biomm.com/informacoes-financeiras/central-de-resultados/',
            'BLAU_FARMACÊUTICA': 'https://ri.blau.com/informacoes-financeiras/central-de-resultados/',
            'BOA_SAFRA': 'https://ri.boasafrasementes.com.br/informacoes-financeiras/central-de-resultados/',
            'BRASKEM': 'https://www.braskem-ri.com.br/divulgacoes-documentos/central-de-resultados/',
            'BRISANET': 'https://ri.brisanet.com.br/informacoes-aos-investidores/central-de-resultados',
            'C&A': 'https://ri.cea.com.br/central-de-resultados/',
            'CAMIL': 'https://ri.camil.com.br/informacoes-financeiras/central-de-resultados/',
            'CBA': 'https://ri.cba.com.br/informacoes-financeiras/central-de-resultados/',
            'CEB': 'https://ri.ceb.com.br/informacoes-aos-investidores/central-de-resultados/',
            'CEEE_D': 'https://ri.equatorialenergia.com.br/informacoes-financeiras/central-de-resultados/',
            'CESP': 'https://ri.aurenenergia.com.br/informacoes-financeiras/central-de-resultados-cesp/',
            'COPEL_UNT_N2': 'https://ri.copel.com/dados-financeiros/central-de-resultados/',
            'CSN': 'https://ri.csn.com.br/informacoes-financeiras/central-de-resultados/',
            'CVC': 'https://www.cvccorp.com.br/investidores/informacoes-financeiras/central-de-resultados/',
            'CELESC': 'https://ri.celesc.com.br/central-de-resultados/',
            'CELULOSE_IRANI': 'https://ri.irani.com.br/servicos-aos-investidores/central-de-resultados/',
            'CEMIG': 'https://ri.cemig.com.br/divulgacao-e-resultados/central-de-resultados',
            'CLEARSALE': 'https://ri.clear.sale/listresultados.aspx?idCanal=DAt3EYDujpS8+xJq08umEw==',
            'COGNA': 'https://ri.cogna.com.br/servico-aos-investidores/arquivos/',
            'COMGÁS': 'https://ri.comgas.com.br/informacoes-financeiras/central-de-resultados/',
            'COMPASS': 'https://www.compassbr.com/divulgacao-e-resultados/central-de-resultados/',
            'COPEL': 'https://ri.copel.com/dados-financeiros/central-de-resultados/',
            'COTEMINAS': 'https://ri.springs.com/informacoes-financeiras/central-de-resultados/',
            'CRUZEIRO_DO_SUL_EDUCACIONAL': 'https://ri.cruzeirodosuleducacional.com.br/informacoes-aos-investidores/central-de-resultados/',
            'CYRELA_REALTY': 'https://ri.cyrela.com.br/informacoes-financeiras/central-de-resultados/',
            'DESKTOP': 'https://www.ri.desktop.com.br/informacoes-financeiras/central-de-resultados/',
            'DEXCO': 'https://ri.dex.co/informacoes-ao-mercado/central-de-resultados/',
            'DEXXOS_PART': 'https://www.dexxos.com.br/informacoes-aos-investidores/central-de-resultados/',
            'DIMED': 'https://ri.grupopanvel.com.br/informacoes-financeiras/central-de-resultados/',
            'DIRECIONAL': 'https://ri.direcional.com.br/informacoes-financeiras/central-de-resultados/',
            'DOTZ': 'https://ri.dotz.com.br/informacoes-financeiras/central-de-resultados/',
            'ELETROMÍDIA': 'https://ri.eletromidia.com.br/informacoes-financeiras/central-de-resultados/',
            'ENJOEI': 'https://ri.enjoei.com/pt-br/informacoes-aos-investidores/central-de-resultados/',
            'ESPAÇOLASER': 'https://ri.espacolaser.com.br/central-de-resultados/arquivos/',
            'EUCATEX': 'https://ri.eucatex.com.br/central-de-resultados',
            'EVEN': 'https://ri.even.com.br/informacoes-financeiras/central-de-resultados/',
            'EZ_TEC': 'https://ri.eztec.com.br/central-de-resultados/',
            'FLEURY': 'https://ri.fleury.com.br/informacoes-financeiras-e-apresentacoes/central-de-resultados/',
            'GAFISA': 'https://ri.gafisa.com.br/informacoes-financeiras/central-resultados/',
            'GERDAU': 'https://ri.gerdau.com/informacoes-ao-mercado/central-de-resultados/',
            'GRUPO_CASAS_BAHIA': 'https://ri.grupocasasbahia.com.br/informacoes-financeiras/central-de-resultados/',
            'GRUPO_JSL': 'https://ri.jsl.com.br/central-de-resultados/',
            'GRUPO_MATEUS': 'https://ri.grupomateus.com.br/central-de-resultados/',
            'GRUPO_SBF': 'https://ri.gruposbf.com.br/informacoes-financeiras/central-de-resultados/',
            'GRUPO_SOMA': 'https://www.somagrupo.com.br/investidores/central-de-resultados/',
            'GRUPOS_GPS': 'https://ri.gpssa.com.br/informacoes-financeiras/central-de-resultados/',
            'HELBOR': 'https://ri.helbor.com.br/central-de-resultados/resultados-trimestrais/',
            'HIDROVIAS_DO_BRASIL': 'https://ri.hbsa.com.br/informacoes-aos-acionistas/central-de-resultados/',
            'HOSPITAL_CARE_CALEDONIA': 'https://ri.hospitalcare.com.br/informacoes-aos-investidores/central-de-resultados/',
            'HYPERA_PHARMA': 'https://ri.hypera.com.br/resultados-e-publicacoes/central-de-resultados/',
            'ISA_CTEEP': 'https://ri.isaenergiabrasil.com.br/pt/informacoes-financeiras/central-de-resultados',
            'INFRACOMMERCE': 'https://ri.infracommerce.com.br/central-de-resultados',
            'INTELBRAS': 'https://ri.intelbras.com.br/informacoes-financeiras/central-de-resultados/',
            'INTERCEMENT': 'https://investidores.intercement.com.br/informacoes-financeiras/central-de-resultados/',
            'JBS': 'https://ri.jbs.com.br/informacoes-financeiras/central-de-resultados/',
            'JHSF': 'https://ri.jhsf.com.br/informacoes-financeiras/central-de-resultados/',
            'JALLES_MACHADO': 'https://ri.jalles.com/central-de-resultados/',
            'KLABIN': 'https://ri.klabin.com.br/divulgacoes-e-resultados/central-de-resultados/',
            'KORA_SAÚDE': 'https://ri.korasaude.com.br/informacoes-financeiras/central-de-resultados/',
            'LIVETECH': 'https://ri.wdcnet.com.br/central-de-resultados',
            'LOCAMERICA': 'https://ri.unidas.com.br/resultados-trimestrais/',
            'LOCAWEB': 'https://ri.lwsa.tech/informacoes-financeiras/central-de-resultados/',
            'LOJAS_RENNER': 'https://lojasrenner.mzweb.com.br/info-aos-investidores/central-de-resultados/',
            'MRV_ENGENHARIA': 'https://ri.mrv.com.br/informacoes-financeiras/central-de-resultados/',
            'MARFRIG': 'https://ri.marfrig.com.br/informacoes-financeiras/central-de-resultados/',
            'MARISA': 'https://ri.marisa.com.br/informacoes-financeiras/central-de-resultados/',
            'MATER_DEI': 'https://ri.materdei.com.br/informacoes-aos-acionistas/central-de-resultados/',
            'METAL_LEVE': 'https://ri.mahle.com.br/informacoes-financeiras/central-de-resultados/',
            'MINERVA': 'https://ri.minervafoods.com/resultados-trimestrais/',
            'MITRE_REALTY': 'https://ri.mitrerealty.com.br/informacoes-financeiras/central-de-resultados/',
            'MOBLY': 'https://investors.grupotoky.com.br/central-de-resultados/',
            'MOURA_DUBEUX': 'https://ri.mouradubeux.com.br/informacoes-financeiras/central-de-resultados/',
            'MOVIDA': 'https://ri.movida.com.br/resultados-trimestrais/',
            'MULTI': 'https://ri.multilaser.com.br/informacoes-financeiras/central-de-resultados/',
            'NEOENERGIA': 'https://ri.neoenergia.com/resultados-e-indicadores/central-de-resultados/',
            'NEOGRID': 'https://ri.neogrid.com/informacoes-aos-investidores/resultados-trimestrais/',
            'OSX_BRASIL': 'https://www.osx.com.br/listresultados.aspx?idCanal=fS5pmXq5hlsF2t3uUGF3gw==',
            'OCEANPACT': 'https://ri.oceanpact.com/informacoes-financeiras/resultados-trimestrais/',
            'ODONTOPREV': 'https://ri.odontoprev.com.br/informacoes-aos-acionistas/central-de-resultados/',
            'ONCOCLÍNICAS': 'https://ri.grupooncoclinicas.com/informacoes-financeiras/central-de-resultados/',
            'ORIZON': 'https://ri.orizonvr.com.br/informacoes-financeiras/central-de-resultados/',
            'PETRORIO': 'https://ri.prio3.com.br/informacoes-financeiras/central-de-resultados/',
            'PAGUE_MENOS': 'https://ri.paguemenos.com.br/informacoes-aos-investidores/central-de-resultados/',
            'PETRORECÔNCAVO_GERAL_SA': 'https://ri.petroreconcavo.com.br/informacoes-financeiras/central-de-resultados/',
            'PETROBRAS': 'https://www.investidorpetrobras.com.br/resultados-e-comunicados/central-de-resultados/',
            'PETZ': 'https://ri.petz.com.br/informacoes-financeiras/central-de-resultados/',
            'PLANO_&_PLANO': 'https://ri.planoeplano.com.br/informacoes-financeiras/central-de-resultados/',
            'POSITIVO_INF': 'https://ri.positivotecnologia.com.br/informacoes-ao-mercado/central-de-resultados/',
            'PÃO_DE_AÇÚCAR': 'https://www.gpari.com.br/informacoes-financeiras/resultados-trimestrais/',
            'QUALICORP': 'https://ri.qualicorp.com.br/informacoes-financeiras/resultados-trimestrais/',
            'QUERO-QUERO': 'https://ri.quero-quero.com.br/informacoes-financeiras/central-de-resultados/',
            'RAÍZEN': 'https://ri.raizen.com.br/informacoes-financeiras/central-de-resultados/',
            'RENOVA': 'https://ri.renovaenergia.com.br/informacoes-financeiras/central-de-resultados/',
            'RIO_ALTO_ENERGIAS_RENOVÁVEIS': 'https://ri.gruporioalto.com.br/informacoes-financeiras/central-de-resultados/',
            'RODOBENS': 'https://ri.rodobens.com.br/informacoes-aos-investidores/central-de-resultados/',
            'ROSSI_RESID': 'https://ri.rossiresidencial.com.br/informacoes-financeiras/central-de-resultados/',
            'SABESP': 'https://ri.sabesp.com.br/informacoes-financeiras/central-de-resultados/',
            'SANEPAR': 'https://ri.sanepar.com.br/informacoes-financeiras/central-de-resultados',
            'SEQUOIA': 'https://ri.sequoialog.com.br/informacoes-financeiras/central-de-resultados/',
            'SER_EDUCACIONAL': 'https://ri.sereducacional.com/informacoes-financeiras/resultados/',
            'SIMPAR': 'https://ri.simpar.com.br/central-de-resultados/',
            'SMART_FIT': 'https://investor.smartfit.com.br/informacoes-financeiras/central-de-resultados/',
            'SUZANO': 'https://ri.suzano.com.br/Portuguese/informacoes-financeiras/central-de-resultados/',
            'TAESA': 'https://ri.taesa.com.br/divulgacao-ao-mercado/central-de-resultados/',
            'TIM_PARTICIPAÇÕES': 'https://ri.tim.com.br/informacoes-ao-mercado/central-de-resultados/',
            'TOTVS': 'https://ri.totvs.com/informacoes-financeiras/central-de-resultados/',
            'TRACK_&_FIELD': 'https://www.tfco.com.br/ri/informacoes-financeiras/central-de-resultados/',
            'TRONOX': 'https://www.tronox-ri.com.br/informacoes-financeiras/central-de-resultados/',
            'ULTRAPAR': 'https://ri.ultra.com.br/divulgacoes-e-resultados/central-de-resultados/',
            'UNIFIQUE': 'https://ri.unifique.com.br/informacoes-financeiras/central-de-resultados/',
            'UNIPAR': 'https://ri.unipar.com/informacoes-aos-investidores/central-de-resultados/',
            'VAMOS': 'https://ri.grupovamos.com.br/informacoes-financeiras/resultados-trimestrais/',
            'VESTE': 'https://www.veste.com/informacoes-financeiras/central-de-resultados/',
            'VIBRA_ENERGIA': 'https://ri.vibraenergia.com.br/divulgacao-e-resultados/central-de-resultados/',
            'VITRU': 'https://investors.vitru.com.br/informacoes-financeiras/central-de-resultados/',
            'VITTIA_FERTILIZANTES': 'https://ri.vittia.com.br/pt/informacoes-financeiras/central-de-resultados',
            'VIVEO': 'https://ri.viveo.com.br/informacoes-financeiras-2/central-de-resultados/'
            }

In [24]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import os
import time
import requests

# Lista de trimestres
trimestres = ["1T24", "2T24", "3T24", "4T24"]

# Configura navegador
options = Options()
options.add_argument("--headless")
driver = webdriver.Chrome(options=options)

# Pasta base
base_dir = os.path.expanduser("/Users/thiagomoncinhatto/Documents/02 - MBA Finanças/TCC/DFs/DFS_TRIMESTRAIS_2024/")

for nome_empresa, url in empresas.items():
    print(100*"-")
    print(f"\n🔍 Processando {nome_empresa}...")
    driver.get(url)
    time.sleep(5)

    # Tenta selecionar o ano 2024, se disponível
    try:
        select_tags = driver.find_elements(By.TAG_NAME, "select")
        found_ano = False
        for sel in select_tags:
            try:
                select_obj = Select(sel)
                options_texts = [opt.text.strip() for opt in select_obj.options]
                if "2024" in options_texts:
                    select_obj.select_by_visible_text("2024")
                    print("📅 Ano 2024 selecionado.")
                    found_ano = True
                    break
            except:
                continue
        if not found_ano:
            print("⚠️ Nenhum seletor de ano com '2024' encontrado.")
    except Exception as e:
        print(f"❌ Erro ao tentar selecionar ano: {e}")

    # Espera o conteúdo principal
    try:
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CLASS_NAME, "first"))
        )
    except Exception as e:
        print(f"⚠️ Conteúdo não carregou para {nome_empresa}: {e}")
        continue

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    empresa_dir = os.path.join(base_dir, nome_empresa)
    os.makedirs(empresa_dir, exist_ok=True)

    encontrou = False
    linhas = soup.find_all('tr')
    for linha in linhas:
        titulo_td = linha.find('td', class_='first titulo')
        if not titulo_td:
            continue
        texto = titulo_td.text.lower()

        if "anuais" in texto or "internacionais" in texto or " em us" in texto:      
            continue
        processar = False
        if "demonstrações financeiras" or "demonstrações contábeis" in texto:
            processar = True

        elif "itr" in texto:
            processar = True

        elif "itr/dfp" in texto:
            processar = True

        if processar:
            icones = linha.find_all('td', class_='icone')
            for idx, icone in enumerate(icones):
                if idx >= len(trimestres):
                    break
                link_tag = icone.find('a', href=True)
                if link_tag:
                    href = link_tag['href']
                    if href.endswith("origin=1") or href.endswith("origin=2"):
                        trimestre = trimestres[idx]
                        filename = f"{nome_empresa}_{trimestre}.pdf"
                        file_path = os.path.join(empresa_dir, filename)

                        print(f"⬇️  Baixando {filename}...")
                        try:
                            pdf_data = requests.get(href).content
                            with open(file_path, 'wb') as f:
                                f.write(pdf_data)
                            print(f"✅ Salvo em: {file_path}")
                        except Exception as e:
                            print(f"❌ Erro ao baixar {href}: {e}")
            encontrou = True
            break

    if not encontrou:
        print(f"⚠️ Nenhuma seção relevante encontrada para {nome_empresa}")

driver.quit()

----------------------------------------------------------------------------------------------------

🔍 Processando 3TENTOS...
📅 Ano 2024 selecionado.
⬇️  Baixando 3TENTOS_1T24.pdf...
✅ Salvo em: /Users/thiagomoncinhatto/Documents/02 - MBA Finanças/TCC/DFs/DFS_TRIMESTRAIS_2024/3TENTOS/3TENTOS_1T24.pdf
⬇️  Baixando 3TENTOS_2T24.pdf...
✅ Salvo em: /Users/thiagomoncinhatto/Documents/02 - MBA Finanças/TCC/DFs/DFS_TRIMESTRAIS_2024/3TENTOS/3TENTOS_2T24.pdf
⬇️  Baixando 3TENTOS_3T24.pdf...
✅ Salvo em: /Users/thiagomoncinhatto/Documents/02 - MBA Finanças/TCC/DFs/DFS_TRIMESTRAIS_2024/3TENTOS/3TENTOS_3T24.pdf
⬇️  Baixando 3TENTOS_4T24.pdf...
✅ Salvo em: /Users/thiagomoncinhatto/Documents/02 - MBA Finanças/TCC/DFs/DFS_TRIMESTRAIS_2024/3TENTOS/3TENTOS_4T24.pdf
----------------------------------------------------------------------------------------------------

🔍 Processando AFLUENTE_T...
📅 Ano 2024 selecionado.
⬇️  Baixando AFLUENTE_T_1T24.pdf...
✅ Salvo em: /Users/thiagomoncinhatto/Documents/

In [26]:
import os

# Base de pastas onde os arquivos foram salvos
base_dir = os.path.expanduser("/Users/thiagomoncinhatto/Documents/02 - MBA Finanças/TCC/DFs/DFS_TRIMESTRAIS_2024/")

# Trimestres esperados
trimestres_esperados = ["1T24", "2T24", "3T24", "4T24"]

# Resultado de empresas com problemas
empresas_incompletas = {}
empresas_ok = []

# Total de empresas
todas_empresas = [nome for nome in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, nome))]
total_empresas = len(todas_empresas)

for nome_empresa in todas_empresas:
    empresa_path = os.path.join(base_dir, nome_empresa)
    
    arquivos = os.listdir(empresa_path)
    pdfs = [f for f in arquivos if f.lower().endswith(".pdf")]

    # Verifica se todos os trimestres estão presentes
    trimestres_encontrados = []
    arquivos_suspeitos = []

    for f in pdfs:
        for trimestre in trimestres_esperados:
            if trimestre in f:
                trimestres_encontrados.append(trimestre)

        # Verificação de integridade: arquivo muito pequeno
        caminho_pdf = os.path.join(empresa_path, f)
        if os.path.getsize(caminho_pdf) < 10 * 1024:  # menos de 10KB
            arquivos_suspeitos.append(f)

    trimestres_faltando = set(trimestres_esperados) - set(trimestres_encontrados)

    if trimestres_faltando or arquivos_suspeitos:
        empresas_incompletas[nome_empresa] = {
            "faltando": list(trimestres_faltando),
            "suspeitos": arquivos_suspeitos
        }
    else:
        empresas_ok.append(nome_empresa)

# Exibe os resultados
print(f"\n📊 Total de empresas processadas: {total_empresas}")
print(f"✅ Empresas com todos os arquivos corretos: {len(empresas_ok)}")
print(f"⚠️ Empresas com problemas: {len(empresas_incompletas)}")

print("\n📌 Empresas OK:")
for emp in sorted(empresas_ok):
    print(f"  ✅ {emp}")

print("\n⚠️ Empresas com problemas:")
for emp, detalhes in empresas_incompletas.items():
    print(f"\n🔻 {emp}")
    if detalhes["faltando"]:
        print(f"  - Trimestres faltando: {', '.join(detalhes['faltando'])}")
    if detalhes["suspeitos"]:
        print(f"  - Arquivos suspeitos (muito pequenos): {', '.join(detalhes['suspeitos'])}")



📊 Total de empresas processadas: 115
✅ Empresas com todos os arquivos corretos: 102
⚠️ Empresas com problemas: 13

📌 Empresas OK:
  ✅ 3TENTOS
  ✅ AFLUENTE_T
  ✅ AGRIBRASIL
  ✅ ALLIAR
  ✅ ALLIED_TECNOLOGIA
  ✅ ALLOS
  ✅ ALPHAVILLE
  ✅ AMBIPAR
  ✅ AREZZO
  ✅ ARMAC
  ✅ AUREN_ENERGIA
  ✅ AUTOMOB
  ✅ AVON
  ✅ BEMOBI
  ✅ BIOMM
  ✅ BLAU_FARMACÊUTICA
  ✅ BOA_SAFRA
  ✅ BRASKEM
  ✅ BR_PARTNERS
  ✅ C&A
  ✅ CBA
  ✅ CEEE_D
  ✅ CELESC
  ✅ CELULOSE_IRANI
  ✅ COMGÁS
  ✅ COMPASS
  ✅ COPEL
  ✅ COPEL_UNT_N2
  ✅ CRUZEIRO_DO_SUL_EDUCACIONAL
  ✅ CSN
  ✅ CVC
  ✅ CYRELA_REALTY
  ✅ DESKTOP
  ✅ DEXCO
  ✅ DEXXOS_PART
  ✅ DIMED
  ✅ DIRECIONAL
  ✅ DOTZ
  ✅ ELETROMÍDIA
  ✅ ENJOEI
  ✅ ESPAÇOLASER
  ✅ EVEN
  ✅ EZ_TEC
  ✅ FLEURY
  ✅ GAFISA
  ✅ GERDAU
  ✅ GRUPOS_GPS
  ✅ GRUPO_JSL
  ✅ GRUPO_MATEUS
  ✅ GRUPO_SBF
  ✅ HELBOR
  ✅ HIDROVIAS_DO_BRASIL
  ✅ HYPERA_PHARMA
  ✅ INTELBRAS
  ✅ INTERCEMENT
  ✅ JALLES_MACHADO
  ✅ JBS
  ✅ JHSF
  ✅ KLABIN
  ✅ LOCAWEB
  ✅ LOJAS_RENNER
  ✅ MARFRIG
  ✅ MARISA
  ✅ MATER_DEI
  ✅ METAL_LEVE
